In [1]:
import torch

In [16]:
t = torch.tensor([[1], [-1], [0], [1], [1], [-1]])
y = torch.tensor([[0.8], [-0.7], [0.1], [0.6], [0.9], [-0.3]])

In [17]:
p_ratio = sum(t == 1) / len(t)

In [6]:
t == 1

tensor([[False,  True],
        [ True, False],
        [False, False],
        [False,  True],
        [False,  True],
        [ True, False]])

In [18]:
p_ratio

tensor([0.5000])

In [8]:
import torch.nn as nn

In [9]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [10]:
output

tensor(1.5866, grad_fn=<NllLossBackward0>)

In [15]:
torch.tensor([1]).squeeze()

tensor(1)